#Define Config


In [ ]:
config = {}
max_uid = 15625

#configuration by param
dataset = "cifar10" #@param {type:"string"} ["cifar10", "cifar100", "ImageNet16"]
run_id =  55# @param {type:"integer"}
trial =  30#@param {type:"integer"}
n_random =  100#@param {type:"integer"}
point = '2a' # @param ['2a', '2b']


config['score'] = 'hook_logdet'
config['nasspace'] = 'nasbench201'
config['augtype'] = 'none'
config['dataset'] = dataset
config['maxofn'] = 3
config['batch_size'] = 128
config['seed'] = 1
config['run_id'] = run_id
config['dataset_id'] = 'CIFAR10'
config['start_uid'] = 0 
config['stop_uid'] =  15000 
config['trial'] = trial
config['n_random'] = n_random
config['point'] = point



#max 15625 stop_uid

#Import NAS Bench API

In [ ]:
%%capture
!git clone https://github.com/MichelePresti/NAS_MachineLearningDeepLearning

In [ ]:
!cp -r /content/NAS_MachineLearningDeepLearning/neural_model .

In [ ]:
import pandas as pd

def get_arch_config_by_dataset(dataset) -> pd.DataFrame:
    """
    This function return the architectures config by dataset in a pandas dataframe.
    PARAMETERS:
       dataset= string among [cifar10, cifar100, imaginet]
    """
    if(dataset == 'cifar10'):
        df = pd.read_csv('/content/NAS_MachineLearningDeepLearning/nas_bench_201__CIFAR10_config.csv', header=0)
        return df
    if(dataset == 'cifar100'):
      df = pd.read_csv('/content/NAS_MachineLearningDeepLearning/nas_bench_201__CIFAR100_config.csv', header=0)
      return df
    if(dataset == 'ImageNet16'):
      df = pd.read_csv('/content/NAS_MachineLearningDeepLearning/nas_bench_201__ImageNet16_config.csv', header=0)
      return df
    else: 
      print('Dataset name not valid')
      return None

def get_standard_config(csv_config: pd.DataFrame) -> dict:
    res = {}
    res['name'] = csv_config.iloc[0]['name']
    res['C'] = csv_config.iloc[0]['C']
    res['N'] = csv_config.iloc[0]['N']
    res['arch_str'] = csv_config.iloc[0]['arch_str']
    res['num_classes'] = 1
    return res
        

In [ ]:
import pandas as pd

searchspace = get_arch_config_by_dataset(config['dataset'])
searchspace


,Unnamed: 0,uid,name,C,N,arch_str,num_classes
0,0,0,infer.tiny,16,5,|avg_pool_3x3~0|+|nor_conv_1x1~0|skip_connect~...,10
1,1,1,infer.tiny,16,5,|nor_conv_3x3~0|+|nor_conv_3x3~0|avg_pool_3x3~...,10
2,2,2,infer.tiny,16,5,|avg_pool_3x3~0|+|nor_conv_3x3~0|nor_conv_3x3~...,10
3,3,3,infer.tiny,16,5,|avg_pool_3x3~0|+|skip_connect~0|none~1|+|none...,10
4,4,4,infer.tiny,16,5,|skip_connect~0|+|skip_connect~0|nor_conv_1x1~...,10
...,...,...,...,...,...,...,...
15620,15620,15620,infer.tiny,16,5,|none~0|+|avg_pool_3x3~0|avg_pool_3x3~1|+|skip...,10
15621,15621,15621,infer.tiny,16,5,|avg_pool_3x3~0|+|nor_conv_3x3~0|nor_conv_3x3~...,10
15622,15622,15622,infer.tiny,16,5,|skip_connect~0|+|nor_conv_3x3~0|nor_conv_3x3~...,10
15623,15623,15623,infer.tiny,16,5,|none~0|+|avg_pool_3x3~0|avg_pool_3x3~1|+|avg_...,10


#Import Dataset

In [ ]:
import torch.utils.data as data
import os, sys
if sys.version_info[0] == 2:
  import cPickle as pickle
else:
  import pickle
class ImageNet16(data.Dataset):
  # http://image-net.org/download-images
  # A Downsampled Variant of ImageNet as an Alternative to the CIFAR datasets
  # https://arxiv.org/pdf/1707.08819.pdf
  
  train_list = [
        ['train_data_batch_1', '27846dcaa50de8e21a7d1a35f30f0e91'],
        ['train_data_batch_2', 'c7254a054e0e795c69120a5727050e3f'],
        ['train_data_batch_3', '4333d3df2e5ffb114b05d2ffc19b1e87'],
        ['train_data_batch_4', '1620cdf193304f4a92677b695d70d10f'],
        ['train_data_batch_5', '348b3c2fdbb3940c4e9e834affd3b18d'],
        ['train_data_batch_6', '6e765307c242a1b3d7d5ef9139b48945'],
        ['train_data_batch_7', '564926d8cbf8fc4818ba23d2faac7564'],
        ['train_data_batch_8', 'f4755871f718ccb653440b9dd0ebac66'],
        ['train_data_batch_9', 'bb6dd660c38c58552125b1a92f86b5d4'],
        ['train_data_batch_10','8f03f34ac4b42271a294f91bf480f29b'],
    ]
  valid_list = [
        ['val_data', '3410e3017fdaefba8d5073aaa65e4bd6'],
    ]

  def __init__(self, root, train, transform, use_num_of_class_only=None):
    self.root      = root
    self.transform = transform
    self.train     = train  # training set or valid set
    if not self._check_integrity(): raise RuntimeError('Dataset not found or corrupted.')

    if self.train: downloaded_list = self.train_list
    else         : downloaded_list = self.valid_list
    self.data    = []
    self.targets = []
  
    # now load the picked numpy arrays
    for i, (file_name, checksum) in enumerate(downloaded_list):
      file_path = os.path.join(self.root, file_name)
      #print ('Load {:}/{:02d}-th : {:}'.format(i, len(downloaded_list), file_path))
      with open(file_path, 'rb') as f:
        if sys.version_info[0] == 2:
          entry = pickle.load(f)
        else:
          entry = pickle.load(f, encoding='latin1')
        self.data.append(entry['data'])
        self.targets.extend(entry['labels'])
    self.data = np.vstack(self.data).reshape(-1, 3, 16, 16)
    self.data = self.data.transpose((0, 2, 3, 1))  # convert to HWC
    if use_num_of_class_only is not None:
      assert isinstance(use_num_of_class_only, int) and use_num_of_class_only > 0 and use_num_of_class_only < 1000, 'invalid use_num_of_class_only : {:}'.format(use_num_of_class_only)
      new_data, new_targets = [], []
      for I, L in zip(self.data, self.targets):
        if 1 <= L <= use_num_of_class_only:
          new_data.append( I )
          new_targets.append( L )
      self.data    = new_data
      self.targets = new_targets
    #    self.mean.append(entry['mean'])
    #self.mean = np.vstack(self.mean).reshape(-1, 3, 16, 16)
    #self.mean = np.mean(np.mean(np.mean(self.mean, axis=0), axis=1), axis=1)
    #print ('Mean : {:}'.format(self.mean))
    #temp      = self.data - np.reshape(self.mean, (1, 1, 1, 3))
    #std_data  = np.std(temp, axis=0)
    #std_data  = np.mean(np.mean(std_data, axis=0), axis=0)
    #print ('Std  : {:}'.format(std_data))
  """
  def __getitem__(self, index):
    img, target = self.data[index], self.targets[index] - 1

    img = Image.fromarray(img)

    if self.transform is not None:
      img = self.transform(img)

    return img, target
  """
  def __len__(self):
    return len(self.data)

  """
  def _check_integrity(self):
    root = self.root
    for fentry in (self.train_list + self.valid_list):
      filename, md5 = fentry[0], fentry[1]
      fpath = os.path.join(root, filename)
      if not check_integrity(fpath, md5):
        return False
    return True
  """


In [ ]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision
import torch

def get_dataset(dataset) -> DataLoader:
    """
    This function return the dataset given its name in torch DataLoader format.
    PARAMETERS:
       dataset= string among [cifar10, cifar100, imaginet]
    """

    if dataset == 'cifar10':
        mean = [x / 255 for x in [125.3, 123.0, 113.9]]
        std = [x / 255 for x in [63.0, 62.1, 66.7]]
        lists = [transforms.RandomHorizontalFlip(), transforms.RandomCrop(32, padding=4), transforms.ToTensor(),
              transforms.Normalize(mean, std)]
        transform = transforms.Compose(lists)
        trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                                download=True, transform=transform)
        train_dt = torch.utils.data.DataLoader(trainset, batch_size=config['batch_size'],
                                                  shuffle=True, num_workers=2)
    elif dataset == 'cifar100':
        mean = [x / 255 for x in [129.3, 124.1, 112.4]]
        std = [x / 255 for x in [68.2, 65.4, 70.4]]
        lists = [transforms.RandomHorizontalFlip(), transforms.RandomCrop(32, padding=4), transforms.ToTensor(),
              transforms.Normalize(mean, std)]
        transform = transforms.Compose(lists)
        trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                                download=True, transform=transform)
        train_dt = torch.utils.data.DataLoader(trainset, batch_size=config['batch_size'],
                                                  shuffle=True, num_workers=2)
    elif dataset.startswith('ImageNet16'):
        mean = [x / 255 for x in [122.68, 116.66, 104.01]]
        std = [x / 255 for x in [63.22, 61.26, 65.09]]
        lists = [transforms.RandomHorizontalFlip(), transforms.RandomCrop(16, padding=2), transforms.ToTensor(),
                 transforms.Normalize(mean, std)]
        transform = transforms.Compose(lists)
        trainset = ImageNet16(config['imaginet_path'], True, transform, 120)
        train_dt = torch.utils.data.DataLoader(trainset, batch_size=config['batch_size'],
                                                  shuffle=True, num_workers=2)
    else:
        raise TypeError("Unknow dataset : {:}".format(dataset))

    return train_dt

In [ ]:
train_dt = get_dataset(config['dataset'])

Files already downloaded and verified


#Define Score

In [ ]:
import numpy as np

def get_batch_jacobian(net, x, target, device, args=None):
    net.zero_grad()
    x.requires_grad_(True)
    y, out = net(x)
    y.backward(torch.ones_like(y))
    jacob = x.grad.detach()
    return jacob, target.detach(), y.detach(), out.detach()


def hooklogdet(K, labels=None):
    s, ld = np.linalg.slogdet(K)
    return ld


def score_network(network, x, x2, target, device):
    jacobs, labels, y, out = get_batch_jacobian(network, x, target, device, config)
    network(x2.to(device))
    value = hooklogdet(network.K, target)
    return value

#Search Algorithm

In [ ]:
import time
import random
from neural_model.neural_model import get_cell_net
from tqdm.notebook import tqdm
from tqdm.contrib.telegram import tqdm, trange

def naswot_search_n2(run_id, dataset_id, device, n):
    result = {'uid': [], 'score': [], 'elapsed_time': []}
    n = config['n_random']
    trial = config['trial']
    point = config['point']
    l = random.sample(range(max_uid), n)
    for uid in tqdm(l,\
       token='5374697833:AAFXE6wumxuSFFwpSzSmEQ6WL4jl8OBVOqw',\
       chat_id='142397010',\
       desc=f'result_run{run_id}_dataset{dataset_id}_point_{point}'):
      #i = uid-start_uid
      net_config: pd.DataFrame = searchspace.loc[searchspace['uid'] == uid]
      net_config: dict = get_standard_config(net_config)
      network = get_cell_net(net_config)
      try:
          start = time.time()
          if 'hook_' in config['score']:

              def counting_forward_hook(module_hook, inp, out):
                  try:
                      if hasattr(module_hook, 'visited_backwards') and not module_hook.visited_backwards:
                          return
                      if isinstance(inp, tuple):
                          inp = inp[0]
                      inp = inp.view(inp.size(0), -1)
                      x = (inp > 0).float()
                      K = x @ x.t()
                      K2 = (1. - x) @ (1. - x.t())
                      if hasattr(network, 'K'):
                        network.K = network.K + K.cpu().numpy() + K2.cpu().numpy()
                      else: 
                        network.K = K.cpu().numpy() + K2.cpu().numpy()
                  except Exception as exception:
                      print(exception)
                      pass


              def counting_backward_hook(module_hook, inp, out):
                  module_hook.visited_backwards = True

              j = []
              for name, module in network.named_modules():
                  j.append(name)
                  if 'ReLU' in str(type(module)):
                      module.register_forward_hook(counting_forward_hook)
                      module.register_backward_hook(counting_backward_hook)

          #print('NUM MODULES IN ARCHITECTURES', len(j))
          # Starting score algorithm
          network = network.to(device)
          ## start time
          #start = time.time()
          random.seed(config['seed'])
          np.random.seed(config['seed'])
          torch.manual_seed(config['seed'])
          s = []
          for j in range(config['maxofn']):
              data_iterator = iter(train_dt)
              x, target = next(data_iterator)
              x2 = torch.clone(x)
              x2 = x2.to(device)
              x, target = x.to(device), target.to(device)
              jacobs, labels, y, out = get_batch_jacobian(network, x, target, device, config)
              if 'hook_' in config['score']:
                  network(x2.to(device))
                  value = hooklogdet(network.K, target)
                  s.append(value)
              else:
                  value = hooklogdet(network.K, target)
                  s.append(value)
          #print(f'Score (uid {uid}): {np.mean(s)}')
          stop = time.time()
          result['uid'].append(uid)
          result['score'].append(np.mean(s))
          result['elapsed_time'].append(stop-start)
          #print(f'Elapsed time (uid {uid}): {stop-start}')
      except Exception as e:
          print(e)
    df = pd.DataFrame.from_dict(result)
    result = {'uid': [], 'score': [], 'elapsed_time': []}
    df.to_csv(f'./result_run{run_id}_{trial}_{dataset_id}.csv')
    
    return 

#2a or 2b

In [ ]:
import pandas as pd

df = pd.read_csv('/content/NAS_MachineLearningDeepLearning/Cifar10Result.csv')
try: 
  df.drop(columns=['Unnamed: 0'], inplace=True)
except:
  print("Already dropped")

acc_df = df


In [ ]:
import time
import random
from neural_model.neural_model import get_cell_net
from tqdm.notebook import tqdm
from tqdm.contrib.telegram import tqdm, trange


def naswot_search_n(run_id, dataset, device, n, trial):
    result = {'uid': [], 'score': [], 'elapsed_time': []}
    n = config['n_random']
    #trial = config['trial']
    #random.seed(i)
    l = random.sample(range(max_uid), n)
    best_score, best_acc, best_net = 0, 0, 0
    best = {}
    best['score'] = 0
    best['acc'] = 0
    best['uid'] = 0
    print(l)
    for uid in tqdm(l,\
       token='5374697833:AAFXE6wumxuSFFwpSzSmEQ6WL4jl8OBVOqw',\
       chat_id='142397010',\
       desc=f'result_run{run_id}_{point}_dataset{dataset_id}_'):
      #i = uid-start_uid
      net_config: pd.DataFrame = searchspace.loc[searchspace['uid'] == uid]
      net_config: dict = get_standard_config(net_config)
      network = get_cell_net(net_config)
      try:
          start = time.time()
          if 'hook_' in config['score']:

              def counting_forward_hook(module_hook, inp, out):
                  try:
                      if hasattr(module_hook, 'visited_backwards') and not module_hook.visited_backwards:
                          return
                      if isinstance(inp, tuple):
                          inp = inp[0]
                      inp = inp.view(inp.size(0), -1)
                      x = (inp > 0).float()
                      K = x @ x.t()
                      K2 = (1. - x) @ (1. - x.t())
                      if hasattr(network, 'K'):
                        network.K = network.K + K.cpu().numpy() + K2.cpu().numpy()
                      else: 
                        network.K = K.cpu().numpy() + K2.cpu().numpy()
                  except Exception as exception:
                      print(exception)
                      pass


              def counting_backward_hook(module_hook, inp, out):
                  module_hook.visited_backwards = True

              j = []
              for name, module in network.named_modules():
                  j.append(name)
                  if 'ReLU' in str(type(module)):
                      module.register_forward_hook(counting_forward_hook)
                      module.register_backward_hook(counting_backward_hook)

          #print('NUM MODULES IN ARCHITECTURES', len(j))
          # Starting score algorithm
          
          network = network.to(device)
          ## start time
          #start = time.time()
          #random.seed(config['seed'])
          #np.random.seed(config['seed'])
          #torch.manual_seed(config['seed'])
          s = []
          for j in range(config['maxofn']):
              data_iterator = iter(train_dt)
              x, target = next(data_iterator)
              x2 = torch.clone(x)
              x2 = x2.to(device)
              x, target = x.to(device), target.to(device)
              jacobs, labels, y, out = get_batch_jacobian(network, x, target, device, config)
              if 'hook_' in config['score']:
                  network(x2.to(device))
                  value = hooklogdet(network.K, target)
                  s.append(value)
              else:
                  value = hooklogdet(network.K, target)
                  s.append(value)
          acc = acc_df['valid-accuracy'].iloc[uid]
          print(f'Score (uid {uid}): {np.mean(s)}, Accuracy: {acc}')
          stop = time.time()
          result['uid'].append(uid)
          result['score'].append(np.mean(s))
          score = np.mean(s)
          result['elapsed_time'].append(stop-start)

          if config['point'] == '2a':
          #point 2a
            if score > best_score:
              best_score = score
              best_net = uid
              best_acc = acc
          else:
            #point 2b
            
            if acc > best_acc:
              best_acc = acc
              best_score = score
              best_net = uid

          #print(f'Elapsed time (uid {uid}): {stop-start}')
      except Exception as e:
          print(e)
    df = pd.DataFrame.from_dict(result)
    result = {'uid': [], 'score': [], 'elapsed_time': []}
    df.to_csv(f'./result_run{run_id}_trial{trial}_{point}_{dataset}.csv')
    print(f'Best Network: {best_net}; Score: {best_score}; Accuracy: {best_acc}')
    best = {}
    best['acc'] = best_acc
    best['score'] = best_score
    best['uid'] = best_net
    series_net = pd.Series(best)
    series_net.to_csv(f'best_network_{run_id}_trial{trial}_{dataset}.csv')
    return 

#All Search

In [ ]:
import time
import random
from neural_model.neural_model import get_cell_net
from tqdm.notebook import tqdm
from tqdm.contrib.telegram import tqdm, trange

def search_all(run_id, dataset_id, device, start_uid, stop_uid):
    result = {'uid': [], 'score': [], 'elapsed_time': []}
    #l = random.sample(range(max_uid), 100)
    #for uid in tqdm(range(start_uid, stop_uid)):
    for uid in tqdm(range(start_uid, stop_uid),\
       token='5374697833:AAFXE6wumxuSFFwpSzSmEQ6WL4jl8OBVOqw',\
       chat_id='142397010',\
       desc=f'result_run{run_id}_{start_uid//1000}_dataset{dataset_id}_'):
      i = uid-start_uid
      net_config: pd.DataFrame = searchspace.loc[searchspace['uid'] == uid]
      net_config: dict = get_standard_config(net_config)
      network = get_cell_net(net_config)
      try:
          start = time.time()
          if 'hook_' in config['score']:

              def counting_forward_hook(module_hook, inp, out):
                  try:
                      if hasattr(module_hook, 'visited_backwards') and not module_hook.visited_backwards:
                          return
                      if isinstance(inp, tuple):
                          inp = inp[0]
                      inp = inp.view(inp.size(0), -1)
                      x = (inp > 0).float()
                      K = x @ x.t()
                      K2 = (1. - x) @ (1. - x.t())
                      if hasattr(network, 'K'):
                        network.K = network.K + K.cpu().numpy() + K2.cpu().numpy()
                      else: 
                        network.K = K.cpu().numpy() + K2.cpu().numpy()
                  except Exception as exception:
                      print(exception)
                      pass


              def counting_backward_hook(module_hook, inp, out):
                  module_hook.visited_backwards = True

              j = []
              for name, module in network.named_modules():
                  j.append(name)
                  if 'ReLU' in str(type(module)):
                      module.register_forward_hook(counting_forward_hook)
                      module.register_backward_hook(counting_backward_hook)

          #print('NUM MODULES IN ARCHITECTURES', len(j))
          # Starting score algorithm
          network = network.to(device)
          ## start time
          #start = time.time()
          random.seed(config['seed'])
          np.random.seed(config['seed'])
          torch.manual_seed(config['seed'])
          s = []
          for j in range(config['maxofn']):
              data_iterator = iter(train_dt)
              x, target = next(data_iterator)
              x2 = torch.clone(x)
              x2 = x2.to(device)
              x, target = x.to(device), target.to(device)
              jacobs, labels, y, out = get_batch_jacobian(network, x, target, device, config)
              if 'hook_' in config['score']:
                  network(x2.to(device))
                  value = hooklogdet(network.K, target)
                  s.append(value)
              else:
                  value = hooklogdet(network.K, target)
                  s.append(value)
          #print(f'Score (uid {uid}): {np.mean(s)}')
          stop = time.time()
          result['uid'].append(uid)
          result['score'].append(np.mean(s))
          result['elapsed_time'].append(stop-start)
          #print(f'Elapsed time (uid {uid}): {stop-start}')
          if i + 1 % 1000 == 0:
            df = pd.DataFrame.from_dict(result)
            result = {'uid': [], 'score': [], 'elapsed_time': []}
            df.to_csv(f'./NASWOT_result_run{run_id}_dataset{dataset_id}_{i}_.csv')
      except Exception as e:
          print(e)
    df = pd.DataFrame.from_dict(result)
    result = {'uid': [], 'score': [], 'elapsed_time': []}
    df.to_csv(f'./result_run{run_id}_{start_uid//1000}_dataset{dataset_id}_LastRecords_.csv')
    
    return 

#Run Algorithm

In [ ]:
def saving_results(trial):
  path = f'./result_run{run_id}_trial{trial}_{point}_{dataset}.csv'
  path_net = f'best_network_{run_id}_trial{trial}_{dataset}.csv'
  
  if point == '2a':
    !cp -r $path_net /content/drive/MyDrive/project/2a/best
    !cp -r $path /content/drive/MyDrive/project/2a/result
  elif point == '2b':
    !cp -r $path_net /content/drive/MyDrive/project/2b/best
    !cp -r $path /content/drive/MyDrive/project/2b/result
  send_requests(trial)
  return


def send_requests(trial):
  TOKEN = '5374697833:AAFXE6wumxuSFFwpSzSmEQ6WL4jl8OBVOqw'
  CHAT_ID = '142397010'
  SEND_URL = f'https://api.telegram.org/bot{TOKEN}/sendMessage'
  your_message = f"RUN COMPLETED!\nInformation:\n\t\t\t\t\t\t\t\tRun_Id:\t{run_id}\n\t\t\t\t\t\t\t\tTrial:{trial}\n\t\t\t\t\t\t\t\tDataset_Id:\t{dataset_id}"
  requests.post(SEND_URL, json={'chat_id': CHAT_ID, 'text': your_message}) 
  return

In [ ]:
import time
import requests
import random
import os
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
os.environ['WANDB_CONSOLE'] = 'off'
start = time.time()

run_id = config['run_id']
dataset = config['dataset']
start_uid = config['start_uid']
stop_uid = config['stop_uid']
n = config['n_random']    # N size of random sample
trial = config['trial']

rand_range = random.sample(range(1000), trial)
print(rand_range)
trial = 0
#search_all(run_id, dataset_id, device, start_uid, stop_uid)
for i in rand_range:
  trial += 1
  config['seed'] = i
  naswot_search_n(run_id, dataset, device, n, trial)
  saving_results(trial)


stop = time.time()

total_time = stop - start

print(f'Total time for search over all searchspace: {total_time}')


[253, 528, 264, 87, 625, 598, 109, 175, 267, 655, 190, 866, 54, 710, 713, 148, 798, 6, 720, 904, 75, 715, 788, 481, 723, 745, 704, 415, 498, 815]
[6194, 11247, 12367, 3065, 1228, 3023, 3009, 10581, 13895, 12654, 3178, 11413, 5006, 5780, 7347, 5381, 9680, 8248, 1254, 4638, 15072, 6761, 6802, 979, 13115, 4729, 3350, 6843, 3336, 5846, 6896, 7267, 14840, 2841, 5757, 2306, 13420, 1288, 9655, 8990, 9257, 5086, 11681, 5556, 12111, 4074, 6520, 14692, 10434, 2226, 11099, 399, 6865, 3042, 7886, 2248, 6474, 8827, 11633, 15498, 9632, 591, 3517, 15604, 5912, 5007, 14862, 3686, 9262, 12546, 6773, 5621, 7387, 15492, 14754, 7743, 4046, 13526, 12753, 12446, 4685, 6675, 680, 1443, 14871, 3246, 7629, 8090, 455, 13997, 9135, 12796, 5744, 4310, 2203, 7239, 4471, 15478, 12086, 6550]


result_run55_2a_datasetCIFAR10_:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1053: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Score (uid 6194): 1721.205078125, Accuracy: 52.36799998901367
Score (uid 11247): 1858.3680419921875, Accuracy: 58.54000000366211
Score (uid 12367): 1439.79638671875, Accuracy: 31.999999997558596
Score (uid 3065): 1664.2523193359375, Accuracy: 55.95599998779297
Score (uid 1228): 1745.6365966796875, Accuracy: 69.30000000976563
Score (uid 3023): 1357.4024658203125, Accuracy: 9.712000000305176
Score (uid 3009): 1818.1063232421875, Accuracy: 68.13599997802734
Score (uid 10581): 1776.1551513671875, Accuracy: 71.36799997802734
Score (uid 13895): 1703.5093994140625, Accuracy: 77.0159999975586
Score (uid 12654): 1728.2034912109375, Accuracy: 64.33999999023438
Score (uid 3178): 1768.6607666015625, Accuracy: 71.64399999267579
Score (uid 11413): 1459.3359375, Accuracy: 55.34400000122071
Score (uid 5006): 1763.9254150390625, Accuracy: 61.127999998779295
Score (uid 5780): 1818.0751953125, Accuracy: 83.11599997558594
Score (uid 7347): 1770.12646484375, Accuracy: 82.57999997070313
Score (uid 5381): 17

result_run55_2a_datasetCIFAR10_:   0%|          | 0/100 [00:00<?, ?it/s]

...

Score (uid 12940): 1856.42431640625, Accuracy: 72.95600000488281
Score (uid 10880): 1769.5594482421875, Accuracy: 71.63200000976562
Score (uid 14043): 1767.3756103515625, Accuracy: 62.66000000244141
Score (uid 9323): 1747.49609375, Accuracy: 67.83599997314452
Score (uid 15135): 1747.2099609375, Accuracy: 69.74800001953125
Score (uid 800): 1812.7020263671875, Accuracy: 67.62399997802734
Score (uid 4724): 1775.349609375, Accuracy: 67.84400001464844
Score (uid 6789): 1809.3841552734375, Accuracy: 74.77199998779297
Score (uid 7427): 1454.3447265625, Accuracy: 40.77599999267578
Score (uid 11406): 1749.947265625, Accuracy: 68.00400001464844
Score (uid 12387): 1702.3507080078125, Accuracy: 74.04400000488282
Score (uid 2512): 1676.583984375, Accuracy: 64.27199998535156
Score (uid 7679): 1672.7391357421875, Accuracy: 66.86799997558593
Score (uid 15405): 1776.8653564453125, Accuracy: 71.85999999267578
Score (uid 11042): 1659.66748046875, Accuracy: 65.83599998413086
Score (uid 12471): 1828.772338

result_run55_2a_datasetCIFAR10_:   0%|          | 0/100 [00:00<?, ?it/s]

Score (uid 14604): 1890.5003662109375, Accuracy: 76.01599998291016
Score (uid 5844): 1653.8817138671875, Accuracy: 67.01599999511718
Score (uid 1615): 1761.404296875, Accuracy: 71.67199997802734
Score (uid 11218): 1818.86474609375, Accuracy: 83.64799997070313
Score (uid 1445): 1768.3011474609375, Accuracy: 80.43599997802734
Score (uid 15011): 1742.708984375, Accuracy: 59.820000004882814
Score (uid 10134): 1489.5335693359375, Accuracy: 73.83200001220703
Score (uid 5036): 1828.31689453125, Accuracy: 68.78800000732421
Score (uid 4626): 1862.3800048828125, Accuracy: 49.8720000024414
Score (uid 6212): 1462.5191650390625, Accuracy: 30.427999995117187
Score (uid 6015): 1816.7547607421875, Accuracy: 66.58
Score (uid 12700): 1766.376953125, Accuracy: 81.40000000732422
Score (uid 316): 1856.1778564453125, Accuracy: 69.71599997070312
Score (uid 10027): 1767.49072265625, Accuracy: 74.33600000732422
Score (uid 3269): 1649.2890625, Accuracy: 56.79599998779297
Score (uid 14540): 1460.865234375, Accur

result_run55_2a_datasetCIFAR10_:   0%|          | 0/100 [00:00<?, ?it/s]

Score (uid 6409): 1462.2327880859375, Accuracy: 53.727999998779296
Score (uid 6495): 1807.6578369140625, Accuracy: 71.04400001220704
Score (uid 9086): 1838.0831298828125, Accuracy: 72.6519999951172
Score (uid 11655): 1817.6474609375, Accuracy: 80.83200001708984
Score (uid 8849): 1858.3345947265625, Accuracy: 75.11599998779297
Score (uid 11380): 1858.1396484375, Accuracy: 68.6399999951172
Score (uid 13213): 1703.88671875, Accuracy: 75.21999998291015
Score (uid 5108): 1855.3656005859375, Accuracy: 81.81199997802734
Score (uid 2597): 1761.2779541015625, Accuracy: 77.13999997314453
Score (uid 11775): 1726.47509765625, Accuracy: 64.72399998779296
Score (uid 10068): 1809.3258056640625, Accuracy: 69.13600000976562
Score (uid 7009): 1859.1737060546875, Accuracy: 73.41599999267578
Score (uid 8489): 1817.7734375, Accuracy: 67.29199998046874
Score (uid 11884): 1794.5859375, Accuracy: 71.11999998046875
Score (uid 12034): 1772.5865478515625, Accuracy: 69.34000000732422
Score (uid 10938): 1727.69433

#Save Result

In [ ]:
#path = f'./result_run{run_id}_{start_uid//1000}_dataset{dataset_id}_LastRecords_.csv'
path = f'./result_run{run_id}_{trial}_{dataset_id}.csv'
path_net = f'best_network_{run_id}_trial{trial}_{dataset_id}.csv'

In [ ]:
!cp -r $path /content/drive/MyDrive/project
!cp -r $path_net /content/drive/MyDrive/project

cp: cannot stat './result_run0_30_CIFAR10.csv': No such file or directory


In [ ]:
%%capture
import requests

TOKEN = '5374697833:AAFXE6wumxuSFFwpSzSmEQ6WL4jl8OBVOqw'
CHAT_ID = '142397010'
SEND_URL = f'https://api.telegram.org/bot{TOKEN}/sendMessage'
your_message = f"RUN COMPLETED!\nInformation:\n\t\t\t\t\t\t\t\tRun_Id:\t{run_id}\n\t\t\t\t\t\t\t\tTrial:{trial}\n\t\t\t\t\t\t\t\tDataset_Id:\t{dataset_id}"
requests.post(SEND_URL, json={'chat_id': CHAT_ID, 'text': your_message}) 